<a href="https://colab.research.google.com/github/enverakbacak/EIR/blob/main/vHashing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/classification/video/Notebooks/

/content/drive/MyDrive/classification/video/Notebooks


In [ ]:
import tensorflow as tf
import numpy as np
import time
import matplotlib
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow.keras.optimizers
from tensorflow.keras import Input 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, TimeDistributed, Conv2D, MaxPool2D, LSTM, Bidirectional, BatchNormalization
from tensorflow.keras.applications import EfficientNetB0, VGG16, ResNet50, MobileNetV2, NASNetMobile
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import pandas as pd
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from tensorflow.keras.optimizers import SGD
import math
from config import Config
from vGenerator import generator
from csv import reader
from tensorflow.keras.utils import to_categorical

In [ ]:
train_data=[]
from csv import reader
# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('../Datasets/UCF101/trainlist01.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    # Check file as empty
    if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            train_data.append(row)
train_data=np.array(train_data)
#train_videos = train_data[0:9520,0]
print(train_data.shape)

#train_labels = train_data[0:9520,1]
#train_labels_onehot = to_categorical(train_labels, dtype ="uint8")
#print(train_labels_onehot.shape)




(9536, 2)


In [ ]:
train_generator = generator(train_data, batch_size=Config.batch_size)
'''
x,y = next(train_generator)
print ('data shape: ', x.shape[:])
print ('label shape: ', y.shape)
print(x.dtype)
'''

"\nx,y = next(train_generator)\nprint ('data shape: ', x.shape[:])\nprint ('label shape: ', y.shape)\nprint(x.dtype)\n"

In [ ]:
video_input = tensorflow.keras.Input(shape = (Config.N, Config.resize,Config.resize,3), name='video')

base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg') # input_shape=(X.shape[1:])
base_model.trainable = False

frame_features = layers.TimeDistributed(base_model)(video_input)
blstm_1   = Bidirectional(LSTM(512, dropout=0.1, recurrent_dropout=0.2, return_sequences = True  ))(frame_features)
blstm_2   = Bidirectional(LSTM(512, dropout=0.1, recurrent_dropout=0.2, return_sequences = False ))(blstm_1)
batchNorm = BatchNormalization()(blstm_2)
Dense_1   = Dense(512)(batchNorm)
Dense_2   = Dense(Config.hash_bits, activation = 'sigmoid' )(Dense_1)
#batchNorm = BatchNormalization()(Dense_2)
Dense_3   = Dense(Config.num_classes, activation='softmax')(Dense_2)
model     = Model(video_input, Dense_3)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 video (InputLayer)          [(None, 10, 224, 224, 3)  0         
                             ]                                   
                                                                 
 time_distributed (TimeDistr  (None, 10, 1280)         2257984   
 ibuted)                                                         
                                                                 
 bidirectional (Bidirectiona  (None, 10, 1024)         7344128   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 1024)             6295552   
 nal)                                                            
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096  

In [ ]:
sgd = SGD(learning_rate=0.01, decay = 1e-6, momentum=0.9, nesterov=True)

def c_loss_1(y_true, y_pred):
    return  ( tf.keras.losses.binary_crossentropy(y_true, y_pred)) 

def c_loss_2(noise_1, noise_2):
    noise_1 = tf.cast(Dense_3 > 0.5 , tf.float32 )
    #noise_1 = tf.cast(np.sign(features) , tf.float32 )
    noise_2 = Dense_3 
    return  ( (1/Config.hash_bits) * tf.keras.metrics.Sum((tf.keras.losses.binary_crossentropy(noise_1, noise_2)))) 

initial_learning_rate = 0.001
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(loss = [c_loss_1, c_loss_2], loss_weights = [0.9, 0.1],  
              optimizer=tensorflow.keras.optimizers.Adam(learning_rate=lr_schedule),
              #optimizer=sgd,
              metrics=['accuracy'],
              )

In [ ]:
cb = [
      tensorflow.keras.callbacks.EarlyStopping(monitor="accuracy", 
                                    patience=15,
                                    mode='min',
                                    verbose=1),
      tensorflow.keras.callbacks.ModelCheckpoint(save_best_only=True, # availabale only for monitor="val_loss"
                                      filepath='./models/model.{epoch:02d}-{loss:.2f}.h5'),
      tensorflow.keras.callbacks.TensorBoard(log_dir='./models/logs'),
     ]


model.fit(train_generator, 
                    steps_per_epoch  = math.ceil(len(train_data) // Config.batch_size), 
                    #validation_data=val_generator,
                    #validation_steps = math.ceil(len(val_data) // batch_size),
                    verbose=1,
                    callbacks=cb, 
                    epochs=Config.num_epochs
                    )

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/5


  1/298 [..............................] - ETA: 1:46:39 - loss: 0.6853 - accuracy: 0.0625

  2/298 [..............................] - ETA: 1:08:11 - loss: 0.6324 - accuracy: 0.0312

  3/298 [..............................] - ETA: 1:08:56 - loss: 0.5826 - accuracy: 0.0521

  4/298 [..............................] - ETA: 1:09:39 - loss: 0.5368 - accuracy: 0.0547

  5/298 [..............................] - ETA: 1:11:16 - loss: 0.4962 - accuracy: 0.0500

  6/298 [..............................] - ETA: 1:11:00 - loss: 0.4589 - accuracy: 0.0521

  7/298 [..............................] - ETA: 1:10:02 - loss: 0.4255 - accuracy: 0.0491

  8/298 [..............................] - ETA: 1:10:34 - loss: 0.3952 - accuracy: 0.0547

  9/298 [..............................] - ETA: 1:10:30 - loss: 0.3689 - accuracy: 0.0521

 10/298 [>.............................] - ETA: 1:10:58 - loss: 0.3449 - accuracy: 0.0562

 11/298 [>.............................] - ETA: 1:10:16 - loss: 0.3238 - accur

In [ ]:
model_json = model.to_json()
with open("models/vHashing_UCF101_512_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("models/vHashing_UCF101_512_weights.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
